In [ ]:
# !pip install transformers datasets peft accelerate bitsandbytes

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
import os

os.environ["WANDB_DISABLED"] = "true"   # disable wandb

# 1. Load model + tokenizer
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="right")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name)

# 2. Apply LoRA config
peft_config = LoraConfig(task_type="CAUSAL_LM", r=8, lora_alpha=16, lora_dropout=0.1)
model = get_peft_model(model, peft_config)

# 3. Tiny dataset with labels
def preprocess(batch):
    tokens = tokenizer(batch["text"], truncation=True, padding="max_length", max_length=64)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

dataset = load_dataset("yelp_polarity", split="train[:200]").map(preprocess, batched=True)

# 4. Training
args = TrainingArguments(
    output_dir="out",
    per_device_train_batch_size=4,
    num_train_epochs=1,
    report_to="none"
)
trainer = Trainer(model=model, args=args, train_dataset=dataset)
trainer.train()

from transformers import pipeline

# Load the fine-tuned model + tokenizer from Trainer
text_generator = pipeline(
    "text-generation",
    model=trainer.model,
    tokenizer=tokenizer,
    device=0  # set -1 if running on CPU
)

# Try with a custom prompt
prompt = "If i would be the Prime Minister of India.."
outputs = text_generator(prompt, max_length=50, num_return_sequences=2, do_sample=True, top_k=50)

# Print results
for i, out in enumerate(outputs):
    print(f"\nGenerated {i+1}: {out['generated_text']}")

